# COMP90051 Project 2

In [ ]:
# Do not edit. These are the only imports permitted.
import numpy as np
from abc import ABC, abstractmethod
from sklearn.tree import DecisionTreeClassifier   # for Task 4
from sklearn.base import clone                    # optional for Task 4
import matplotlib.pyplot as plt                   # for Task 5
from sklearn.metrics.pairwise import rbf_kernel   # for Task 6

In [ ]:
class MAB(ABC):
    """Base class for a contextual multi-armed bandit (MAB)
    
    Parameters
    ----------
    n_arms : int
        Number of arms.
    """
    def __init__(self, n_arms):
        if not np.issubdtype(type(n_arms), np.integer):
            raise TypeError("`n_arms` must be an integer")
        if not n_arms >= 0:
            raise ValueError("`n_arms` must be non-negative")
        self.n_arms = n_arms
        # your code here (if you like)
        
    @abstractmethod
    def play(self, pround,  context):
        """Play a round
        
        Parameters
        ----------     
        pround : play round
           
        context : float numpy.ndarray, shape (n_arms, n_dims), optional
            An array of context vectors presented to the MAB. The 0-th 
            axis indexes the arms, and the 1-st axis indexes the features.
            Non-contextual bandits accept a context of None.
        
        Returns
        -------
        arm : int
            Integer index of the arm played this round. Should be in the set 
            {0, ..., n_arms - 1}.
        """
        # your code here (if you like)
    
    @abstractmethod
    def update(self, arm, reward, context):
        """Update the internal state of the MAB after a play
        
        Parameters
        ----------
        arm : int
            Integer index of the played arm in the set {0, ..., n_arms - 1}.
        
        reward : float
            Reward received from the arm.
        
        context : float numpy.ndarray, shape (n_arms, n_dims), optional
            An array of context vectors that was presented to the MAB. The 
            0-th axis indexes the arms, and the 1-st axis indexes the 
            features. Non-contextual bandits accept a context of None. 
        """
        # your code here (if you like)

In [ ]:
# Define global functions here, if required


## 1. Implement ε-greedy and UCB MABs

In [ ]:
class EpsGreedy(MAB):
    """Epsilon-Greedy multi-armed bandit

    Parameters
    ----------
    n_arms : int
        Number of arms

    epsilon : float
        Explore probability. Must be in the interval [0, 1].

    Q0 : float, default=np.inf
        Initial value for the arms.
    """
    def __init__(self, n_arms, epsilon, Q0=np.inf):
        super().__init__(n_arms)
        # your code here
        self.epsilon = epsilon
        self.n_arms = n_arms  #Number of arms
        self.Q = np.full(n_arms, Q0)  #Reward : Init w/ Q0
        self.pullCnt = np.zeros(n_arms)  #for each arm
        self.totRewards = np.zeros(n_arms)  #reward from each arm
        
    def play(self, pround, context=None):
        super().play(context)
        # your code here
        # Exploitation - pick the best expectation
        if np.random.random() > self.epsilon:
            best_arms = np.argwhere(self.Q == np.amax(self.Q)).flatten().tolist()
            arm_played = np.random.choice(best_arms) #tie case
        
        # Exploration - pick random
        else:
            arm_played = np.random.randint(self.n_arms)  #randome int among arms
        
        return arm_played + 1
        
        
    def update(self, arm, reward, context=None):
        super().update(arm, reward, context)
        # your code here
        idx_arm = arm - 1
        #cal Q
        self.totRewards[idx_arm] += reward
        self.pullCnt[idx_arm] += 1
        cnt = self.pullCnt[idx_arm]
        
        if (cnt>0) : self.Q[idx_arm] = self.totRewards[idx_arm] / cnt
        else : print("update : cnt <= 0")

In [ ]:
class UCB(MAB):
    """Upper Confidence Bound (UCB) multi-armed bandit

    Parameters
    ----------
    n_arms : int
        Number of arms.

    rho : float
        Positive real explore-exploit parameter.

    Q0 : float, default=np.inf
        Initial value for the arms.
    """
    def __init__(self, n_arms, rho, Q0=np.inf):
        super().__init__(n_arms)
        # your code here
        self.rho = rho  # Exploration hyper parameter
        self.Mu = np.zeros(n_arms)  # avg reward of each arm
        
        self.n_arms = n_arms
        self.Q = np.full(n_arms, Q0)
        self.pullCnt = np.zeros(n_arms)
        self.totRewards = np.zeros(n_arms)
    
    def play(self, pround, context=None):
        super().play(context)
        # your code here
        # Update expected reward Q for each arm.
        for idx_arm in range(self.n_arms):            
            cnt = self.pullCnt[idx_arm]
            if cnt > 0:  
                self.Q[idx_arm] = self.Mu[idx_arm] + np.sqrt(self.rho * np.log(pround) / cnt)
        #Pick the best arm w/ highest Q
        best_arms = np.argwhere(self.Q == np.amax(self.Q)).flatten().tolist()
        arm_played = np.random.choice(best_arms)
        return arm_played + 1
        
    def update(self, arm, reward, context=None):
        super().update(arm, reward, context)
        # your code here
        
        # Update Mu w/ avg(observed reward)
        idx_arm = arm - 1
        self.totRewards[idx_arm] += reward
        self.pullCnt[idx_arm] += 1
        
        cnt = self.pullCnt[idx_arm]
        rwds = self.totRewards[idx_arm]

        if cnt > 0:        
            self.Mu[idx_arm] = rwds / cnt
        else : print("update : cnt <= 0")

## 2. Implement off-policy evaluation

In [ ]:


def offlineEvaluate(mab, arms, rewards, contexts, n_rounds=None):
    """Offline evaluation of a multi-armed bandit
    
    Parameters
    ----------
    mab : instance of MAB
        MAB to evaluate.
    
    arms : integer numpy.ndarray, shape (n_events,) 
        Array containing the history of pulled arms, represented as integer 
        indices in the set {0, ..., mab.n_arms}
    
    rewards : float numpy.ndarray, shape (n_events,)
        Array containing the history of rewards.
    
    contexts : float numpy.ndarray, shape (n_events, n_arms, n_dims)
        Array containing the history of contexts presented to the arms. 
        The 0-th axis indexes the events in the history, the 1-st axis 
        indexes the arms and the 2-nd axis indexed the features.
        
    n_rounds : int, default=None
        Number of matching events to evaluate the MAB on. If None, 
        continue evaluating until the historical events are exhausted.
    
    Returns
    -------
    out : float numpy.ndarray
        Rewards for the matching events.
    """
    # your code here
    tround = 0
    R = []  # initial payoffs is empty
    # Note: history object as mentioned in the original algorithm is not maintained as an object here
    #       as it is not used in our implementation
    
    # If nrounds is not set, will go through all logs from the given datasets
    if n_rounds is None:
        n_rounds = len(dataset)
    
    for log in range(len(dataset)):  #step through the stream of logged events one by one
        
        # Once function finds (nrounds) matching arm plays, it should stop
        if tround >= n_rounds:
            break
            
        context = contexts[log].reshape(10,10) # convert 1D flat context to (n_arms X ndims) 2D matrix
        
        # pull an arm with context provided if applicable
        played_arm = mab.play(tround + 1, context)
        
        # if the arm played matches a given log
        # note down the reward as if the bandit really received this reward
        # and update the bandit with the played arm, reward, and context
        # and increment supplied tround number only with the length of history recorded so far
        if played_arm == arms[log]:
            R.append(rewards[log])  
            mab.update(played_arm, rewards[log], context)  
            tround += 1 
        
    return R

In [ ]:
# load dataset here
dataset = np.loadtxt('dataset.txt')
arms = dataset[:,0].astype(int)  # Column 1
rewards = dataset[:,1].astype(int)  # Column 2
contexts = dataset[:,2:].astype(int)  # Column 3:102

In [ ]:
mab = EpsGreedy(10, 0.05)
results_EpsGreedy = offlineEvaluate(mab, arms, rewards, contexts, 800)
print('EpsGreedy average reward', np.mean(results_EpsGreedy))

In [ ]:
mab = UCB(10, 1.0)
results_UCB = offlineEvaluate(mab, arms, rewards, contexts, 800)
print('UCB average reward', np.mean(results_UCB))

## 3. Implement LinUCB contextual MAB

In [ ]:
class LinUCB(MAB):
    """Contextual multi-armed bandit (LinUCB)

    Parameters
    ----------
    n_arms : int
        Number of arms.

    n_dims : int
        Number of features for each arm's context.

    alpha : float
        Positive real explore-exploit parameter.
    """
    def __init__(self, n_arms, n_dims, alpha):
        super().__init__(n_arms)
        # your code here
    
    def play(self, context):
        super().play(context)
        # your code here
    
    def update(self, arm, reward, context):
        super().update(arm, reward, context)
        # your code here

In [ ]:
mab = LinUCB(10, 10, 1.0)
results_LinUCB = offlineEvaluate(mab, arms, rewards, contexts, 800)
print('LinUCB average reward', np.mean(results_LinUCB))

## 4. Implement TreeBootstrap contextual MAB

In [ ]:
class TreeBootstrap(MAB):
    """Contextual Thompson sampled multi-armed bandit (TreeBootstrap)

    Parameters
    ----------
    n_arms : int
        Number of arms.

    n_dims : int
        Number of features for each arm's context.

    tree : instance of sklearn.tree.DecisionTreeClassifier, optional
        Decision tree to use for predicting the expected future reward. 
        Defaults to sklearn.tree.DecisionTreeClassifier().
    """
    def __init__(self, n_arms, n_dims, tree=DecisionTreeClassifier()):
        super().__init__(n_arms)
        # your code here
        
    def play(self, context):
        super().play(context)
        # your code here
    
    def update(self, arm, reward, context):
        super().update(arm, reward, context)
        # your code here

In [ ]:
mab = TreeBootstrap(10, 10)
results_TreeBootstrap = offlineEvaluate(mab, arms, rewards, contexts, 800)
print('TreeBootstrap average reward', np.mean(results_TreeBootstrap))

## 5. Evaluation and hyperparameter tuning for LinUCB
### 5.A.

In [ ]:
# your code here

### 5.B.

In [ ]:
# your code here

## 6. Implement KernelUCB contextual MAB

In [ ]:
class KernelUCB(MAB):
    """Kernelised contextual multi-armed bandit (Kernelised LinUCB)
    
    Parameters
    ----------
    n_arms : int
        Number of arms.

    n_dims : int
        Number of features for each arm's context.

    gamma : float
        Positive real explore-exploit parameter.
    
    eta : float
        Positive real explore-exploit parameter.
    
    kern : callable
        A kernel function from sklearn.metrics.pairwise.
    """
    def __init__(self, n_arms, n_dims, gamma, eta, kern):
        super().__init__(n_arms)
        # your code here
        
    def play(self, context):
        super().play(context)
        # your code here
    
    def update(self, arm, reward, context):
        super().update(arm, reward, context)
        # your code here

In [ ]:
mab = KernelUCB(10, 10, 1.0, 0.1, rbf_kernel)
results_KernelUCB = offlineEvaluate(mab, arms, rewards, contexts, 800)
print('KernelUCB average reward', np.mean(results_KernelUCB))

In [ ]:
# your plotting code here